RNEC Scraper

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

%run "C:\Users\Diogo\OneDrive - NOVAIMS\My Functions\TG_message.py"

#Field codes of interest
fields = ['t83','t87','t91','t95','t99','t103','t107','t109','t115','t119','t123','t129','t134','t158','t164','t170']

# Df Cols
keys = ['RNEC Number',
        'EudraCT Number',
        'Protocol Number',
        'Type of Study',
        "Sponsor's Name",
        'Title of the trial',
        'Recruitment Status',
        'Date of Global End',
        'Infarmed Authorization Date',
        'CEC Opinion Date',
        'Non-technical title',
        'Paediatric Investigation Plan',
        'Medical condition(s)',
        'Therapeutic Confirmatory (Phase II)',
        'Therapeutic Confirmatory (Phase III)',
        'Therapeutic Confirmatory (Phase IV)',
        'Age_Range',
        'Gender'
        ]


# Temp list to hold the scraped data
data = []
# Final df to merge data into
df = pd.DataFrame(columns=keys)

# Initialization 
driver = webdriver.Chrome()
# Goes to target page
driver.get("https://www.rnec.pt/pesquisa-de-estudos-clinicos")
# Clicks button to make an empty search (load all trials) 
search_button = driver.find_element(By.ID, '_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt73').click()
driver.implicitly_wait(15)
# Goes to the first page 
page_1_update = driver.find_element(By.XPATH, '//*[@id="_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt79_paginator_bottom"]/span[3]/span[1]').click()


'''
For every page selenium gets the html code and bs parse it and find the relevant fields and get it's values. For age and gender is targets the parent field 
and gets text from sibling. There's only 2 trials per page that get stored in a temp list (study_data) that passes the data to another (data) outside the loop 
and resets each iteration. In each iteration, it's added 2 to the incrementor because each element of interest also contains the trial number, as well
as an id(obtained iterating fields list). At the end of the loop a next page button is clicked and the driver set to wait 5 sec to ensure data is correctly retrieved. 
Try blocks were implemented because field existence was dynamic, some trials don't have certain fields.
'''
# Trial number incrementor
study_num = (0, 1)
try:
    for page in range (1,299):
        page_html = driver.page_source
        soup = BeautifulSoup(page_html, 'html.parser')
        for study in study_num:
            study_data = []
            for field in fields:
                try:
                    element = soup.find(id=f'_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt79:{study}:j_id{field}')['value']       
                except:
                    element = None
                study_data.append(element)
            try:
                age_range= soup.find('label', {'id': f'_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt79:{study}:j_idt173'}).parent.find_next_sibling().text.strip()
                gender = soup.find('label', {'id': f'_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt79:{study}:j_idt181'}).parent.find_next_sibling().text.strip()
            except:
                age_range = None
                gender = None
            study_data.append(age_range)
            study_data.append(gender)
            data.append(study_data)
        
        study_num = tuple([x + 2 for x in study_num])
        next_page = driver.find_element(By.XPATH, '//*[@id="_RNECPesquisaEstudosClinicos_WAR_RNECPesquisaEstudosClinicosportlet_:mainForm:j_idt79_paginator_bottom"]/span[4]/span').click()
        time.sleep(5)
        
        
    dict = [{k:v for k,v in zip(keys, d)} for d in data]
    df = df.append(dict, ignore_index=True)
    df.to_csv('../DataSets/RNEC.csv', index=False)
    driver.quit()
    message('Success!')
except:
    message('ERROR')


C:\Users\Diogo\AppData\Local\Temp\ipykernel_34036\2854995103.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict, ignore_index=True)
